### Some initial setup for Spark to access Cassandra tables

In [2]:
import json
import time
import pytz
import traceback
import time_uuid
from pytz import timezone
from datetime import datetime
from pyspark.sql import types
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SQLContext, Row
from pyspark import SparkContext, SparkConf
from config import *

In [3]:
sc.stop()

In [4]:
conf = SparkConf()\
    .setAppName(APPNAME)\
    .setMaster(MASTER)\
    .set("spark.cassandra.connection.host", CASSANDRA_HOST)\
    .set("spark.cassandra.connection.port", CASSANDRA_PORT)\
    .set("spark.cassandra.auth.username", CASSANDRA_USERNAME)\
    .set("spark.cassandra.auth.password", CASSANDRA_PASSWORD)

In [5]:
sc = SparkContext(MASTER, APPNAME, conf=conf)
sqlContext = SQLContext(sc)

In [6]:
sqlContext.sql("""CREATE TEMPORARY TABLE %s \
                  USING org.apache.spark.sql.cassandra \
                  OPTIONS ( table "%s", \
                            keyspace "%s", \
                            cluster "Test Cluster", \
                            pushdown "true") \
              """ % (TABLE_EDGES, TABLE_EDGES, KEYSPACE))

sqlContext.sql("""CREATE TEMPORARY TABLE %s \
                  USING org.apache.spark.sql.cassandra \
                  OPTIONS ( table "%s", \
                            keyspace "%s", \
                            cluster "Test Cluster", \
                            pushdown "true") \
              """ % (TABLE_VERTICES, TABLE_VERTICES, KEYSPACE))

DataFrame[]

### Following are the tables available to query from Pyspark

In [7]:
print TABLE_EDGES
print TABLE_VERTICES

graph_edges
graph_vertices


### Some queries on the table "graph_edges"

In [12]:
sqlContext.sql('SELECT * FROM graph_edges LIMIT 5').show()

+-------------------+--------------------+----------------+------------+------------+-------+-------+-------------------+
|comment_mobile_orig|          updated_at|comment_duration|comment_type|contact_name|    dst|    src|tagged_contact_name|
+-------------------+--------------------+----------------+------------+------------+-------+-------+-------------------+
|         8981846923|2017-02-12 09:06:...|              39|       Type4| Lisa Wilson|177_322| 92_316|     Candice Tucker|
|         8981846923|2017-01-18 12:15:...|              39|       Type4|Mike Jenkins|177_322| 92_316|      Anne Mckinney|
|         8981846923|2016-09-21 20:08:...|              39|       Type4| Bryan Salas|177_322| 92_316|     Brittany Lucas|
|         8981846923|2016-06-13 14:02:...|              39|       Type4| Robert Bond|177_322| 92_316|   Christine Howard|
|         7754838671|2017-02-22 18:01:...|              50|        Spam|Shawn Pierce|466_446|145_362|          John Pugh|
+-------------------+---

In [13]:
sqlContext.sql('SELECT comment_mobile_orig, COUNT(*) AS count FROM graph_edges GROUP BY comment_mobile_orig ORDER BY count DESC LIMIT 5').show()

+-------------------+-----+
|comment_mobile_orig|count|
+-------------------+-----+
|         8904758624|    4|
|         7316154827|    4|
|         7361088427|    4|
|         9631179824|    4|
|         8722552424|    4|
+-------------------+-----+



### Some queries on the table "graph_vertices"

In [14]:
sqlContext.sql('SELECT * FROM graph_vertices LIMIT 5').show()

+-------+------------+
|     id|mobilenumber|
+-------+------------+
|185_376|  8990126137|
|494_383|  8767285455|
|397_235|  8676721437|
|251_293|  8710790829|
| 64_463|  7134881438|
+-------+------------+



In [15]:
sqlContext.sql('SELECT mobilenumber, COUNT(*) AS count FROM graph_vertices GROUP BY mobilenumber ORDER BY count DESC LIMIT 5').show()

+------------+-----+
|mobilenumber|count|
+------------+-----+
|  7913279024|    1|
|  8360734624|    1|
|  7235858824|    1|
|  9031885427|    1|
|  9575982027|    1|
+------------+-----+

